# IMPORT FUNCFIONS

In [1]:
import pandas as pd
import numpy as np
import networkx as nx
import itertools
import pickle
import os
import random
from sklearn.model_selection import train_test_split

from sklearn.metrics import average_precision_score
from sklearn import metrics

from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression

from ge import LINE

# READ DATA

In [2]:
def get_edges_dict(traindata):
    if os.path.isfile("edgesdic.pkl"):
        f = open("edgesdic.pkl","rb")
        edges = pickle.load(f)
        f.close()
        return edges
    else:
        edges = dict()
        for edge in range(len(traindata)): 
            edges[(traindata.loc[[edge]]['id_1'].values[0],traindata.loc[[edge]]['id_2'].values[0])] = 1 
            edges[(traindata.loc[[edge]]['id_2'].values[0],traindata.loc[[edge]]['id_1'].values[0])] = 1  
        f = open("edgesdic.pkl","wb")
        pickle.dump(edges,f)
        f.close()
        return edges

def get_negative_edges(traindata,g):
    edges = get_edges_dict(traindata)
    # for missing edges.
    negative_edges = set([])
    maxNodenum = max(max(traindata['id_1']),max(traindata['id_2']))
    #產生與原圖同edge數目的negative edges
    while (len(negative_edges)<len(traindata)):
        node1=random.randint(0, maxNodenum) 
        node2=random.randint(0, maxNodenum) 
        tmp = edges.get((node1,node2),0) #edge不在graph裡標為0
        if tmp == 0 and node1!=node2 and g.has_node(node1) and g.has_node(node2): # if edge不在graph裡才要做處理
            negative_edges.add((node1,node2))
        else:
            continue
    return negative_edges

# MODELS

In [3]:
def combine_embedding(data,embeddings):
    i=0
    X = []
    for node1,node2 in data:
        X.append(np.concatenate((data[i],embeddings[int(node1)],embeddings[int(node2)])))
   # print(embeddings[str(int(data[0]))])
        i+=1
    return X

In [4]:
all_ap = []
all_auc = []

filename = 'wiki'

for i in range(10):
    ori_df = pd.read_csv('data/'+filename+'.txt', header=None, sep=' ')
    ori_df.columns = ['id_1', 'id_2']

    ori_G = nx.from_pandas_edgelist(ori_df, 'id_1', 'id_2')
    #print(nx.info(ori_G))
    
    negative_edges = get_negative_edges(ori_df, ori_G)
    #print(len(negative_edges))

    train_df, test_df = train_test_split(ori_df, test_size=0.1)
    
    nodes_number = len(ori_G.nodes)
    graph_np = np.zeros((nodes_number, nodes_number))
    for i in range(train_df.shape[0]):
        graph_np[train_df.iloc[i, 0], train_df.iloc[i, 1]] = 1
        graph_np[train_df.iloc[i, 1], train_df.iloc[i, 0]] = 1

    train_G = nx.from_numpy_matrix(graph_np)
    #print(nx.info(train_G))
    
    df_neg = pd.DataFrame(list(negative_edges), columns=['id_1', 'id_2'])
    
    train_df = train_df.values
    test_pos_df = test_df.values
    df_neg = df_neg.values
    
    model = LINE(train_G, embedding_size=12, order='all')
    model.train(batch_size=128, epochs=50, verbose=2)
    
    LINE_embeddings = model.get_embeddings()
    
    train_df = combine_embedding(train_df, LINE_embeddings)
    test_pos_df = combine_embedding(test_pos_df, LINE_embeddings)
    df_neg = combine_embedding(df_neg, LINE_embeddings)
    
    X_train_neg, X_test_neg = train_test_split(df_neg, test_size=0.1)
    
    y_train_pos = np.ones(len(train_df))
    y_train_neg = np.zeros(len(X_train_neg))
    y_test_pos = np.ones(len(test_pos_df))
    y_test_neg = np.zeros(len(X_test_neg))
    
    X_train = np.concatenate((train_df, X_train_neg))
    y_train = np.concatenate((y_train_pos, y_train_neg))
    X_test = np.concatenate((test_pos_df, X_test_neg))
    y_test = np.concatenate((y_test_pos, y_test_neg))
    
    clf = RandomForestClassifier(n_estimators=400)
    clf.fit(X_train, y_train)
    
    predict_Y = clf.predict(X_test)
    
    ap = average_precision_score(y_test, predict_Y)
    print("AP： ", ap)
    fpr, tpr, _ = metrics.roc_curve(y_test, predict_Y, pos_label=1)
    auc = metrics.auc(fpr, tpr)
    print("AUC SCORE: ",auc)
    
    all_ap.append(ap)
    all_auc.append(auc)

print("####################### SUM UP ############################")
print("AP MEAN : ", np.array(all_ap).mean())
print("AP STD : ", np.array(all_ap).std())
print("AUC MEAN : ", np.array(all_auc).mean())
print("AUC STD : ", np.array(all_auc).std())

Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor
Instructions for updating:
If using Keras pass *_constraint arguments to layers.
Instructions for updating:
keep_dims is deprecated, use keepdims instead
Epoch 1/50
558/558 - 1s - loss: 1.3859 - first_order_loss: 0.6930 - second_order_loss: 0.6929
Epoch 2/50
558/558 - 1s - loss: 1.3525 - first_order_loss: 0.6923 - second_order_loss: 0.6602
Epoch 3/50
558/558 - 1s - loss: 1.2262 - first_order_loss: 0.6898 - second_order_loss: 0.5363
Epoch 4/50
558/558 - 1s - loss: 1.1331 - first_order_loss: 0.6855 - second_order_loss: 0.4477
Epoch 5/50
558/558 - 1s - loss: 1.0862 - first_order_loss: 0.6807 - second_order_loss: 0.4055
Epoch 6/50
558/558 - 1s - loss: 1.0575 - first_order_loss: 0.6760 - second_order_loss: 0.3815
Epoch 7/50
558/558 - 1s - loss: 1.0400 - first_order_loss: 0.6722 - second_order_loss: 0.3678
Epoch 8/50
558/558 - 1s - loss: 1.0233 - first_order_loss: 0.6682 - se

Epoch 26/50
558/558 - 1s - loss: 0.8712 - first_order_loss: 0.6442 - second_order_loss: 0.2271
Epoch 27/50
558/558 - 1s - loss: 0.8617 - first_order_loss: 0.6435 - second_order_loss: 0.2181
Epoch 28/50
558/558 - 1s - loss: 0.8537 - first_order_loss: 0.6422 - second_order_loss: 0.2115
Epoch 29/50
558/558 - 1s - loss: 0.8449 - first_order_loss: 0.6418 - second_order_loss: 0.2031
Epoch 30/50
558/558 - 1s - loss: 0.8394 - first_order_loss: 0.6423 - second_order_loss: 0.1971
Epoch 31/50
558/558 - 1s - loss: 0.8311 - first_order_loss: 0.6411 - second_order_loss: 0.1900
Epoch 32/50
558/558 - 1s - loss: 0.8250 - first_order_loss: 0.6410 - second_order_loss: 0.1840
Epoch 33/50
558/558 - 1s - loss: 0.8157 - first_order_loss: 0.6386 - second_order_loss: 0.1771
Epoch 34/50
558/558 - 1s - loss: 0.8121 - first_order_loss: 0.6406 - second_order_loss: 0.1715
Epoch 35/50
558/558 - 1s - loss: 0.8077 - first_order_loss: 0.6408 - second_order_loss: 0.1670
Epoch 36/50
558/558 - 1s - loss: 0.8024 - first_or

Epoch 12/50
557/557 - 1s - loss: 0.9854 - first_order_loss: 0.6576 - second_order_loss: 0.3278
Epoch 13/50
557/557 - 1s - loss: 0.9773 - first_order_loss: 0.6555 - second_order_loss: 0.3218
Epoch 14/50
557/557 - 1s - loss: 0.9698 - first_order_loss: 0.6548 - second_order_loss: 0.3150
Epoch 15/50
557/557 - 1s - loss: 0.9606 - first_order_loss: 0.6534 - second_order_loss: 0.3072
Epoch 16/50
557/557 - 1s - loss: 0.9510 - first_order_loss: 0.6514 - second_order_loss: 0.2996
Epoch 17/50
557/557 - 1s - loss: 0.9429 - first_order_loss: 0.6511 - second_order_loss: 0.2918
Epoch 18/50
557/557 - 1s - loss: 0.9320 - first_order_loss: 0.6502 - second_order_loss: 0.2818
Epoch 19/50
557/557 - 1s - loss: 0.9229 - first_order_loss: 0.6477 - second_order_loss: 0.2752
Epoch 20/50
557/557 - 1s - loss: 0.9154 - first_order_loss: 0.6472 - second_order_loss: 0.2682
Epoch 21/50
557/557 - 1s - loss: 0.9035 - first_order_loss: 0.6457 - second_order_loss: 0.2578
Epoch 22/50
557/557 - 1s - loss: 0.8950 - first_or

Epoch 48/50
557/557 - 1s - loss: 0.7606 - first_order_loss: 0.6400 - second_order_loss: 0.1206
Epoch 49/50
557/557 - 1s - loss: 0.7554 - first_order_loss: 0.6374 - second_order_loss: 0.1180
Epoch 50/50
557/557 - 1s - loss: 0.7520 - first_order_loss: 0.6373 - second_order_loss: 0.1147
AP：  0.8813037183822079
AUC SCORE:  0.9024458032240134
Epoch 1/50
559/559 - 2s - loss: 1.3852 - first_order_loss: 0.6930 - second_order_loss: 0.6921
Epoch 2/50
559/559 - 1s - loss: 1.3343 - first_order_loss: 0.6924 - second_order_loss: 0.6419
Epoch 3/50
559/559 - 1s - loss: 1.2069 - first_order_loss: 0.6906 - second_order_loss: 0.5164
Epoch 4/50
559/559 - 1s - loss: 1.1275 - first_order_loss: 0.6867 - second_order_loss: 0.4408
Epoch 5/50
559/559 - 1s - loss: 1.0851 - first_order_loss: 0.6819 - second_order_loss: 0.4032
Epoch 6/50
559/559 - 1s - loss: 1.0583 - first_order_loss: 0.6765 - second_order_loss: 0.3818
Epoch 7/50
559/559 - 1s - loss: 1.0437 - first_order_loss: 0.6720 - second_order_loss: 0.3717
Ep

Epoch 34/50
556/556 - 2s - loss: 0.8108 - first_order_loss: 0.6406 - second_order_loss: 0.1702
Epoch 35/50
556/556 - 2s - loss: 0.8032 - first_order_loss: 0.6387 - second_order_loss: 0.1644
Epoch 36/50
556/556 - 2s - loss: 0.7982 - first_order_loss: 0.6396 - second_order_loss: 0.1586
Epoch 37/50
556/556 - 2s - loss: 0.7933 - first_order_loss: 0.6394 - second_order_loss: 0.1539
Epoch 38/50
556/556 - 2s - loss: 0.7896 - first_order_loss: 0.6378 - second_order_loss: 0.1518
Epoch 39/50
556/556 - 2s - loss: 0.7837 - first_order_loss: 0.6386 - second_order_loss: 0.1451
Epoch 40/50
556/556 - 2s - loss: 0.7817 - first_order_loss: 0.6388 - second_order_loss: 0.1429
Epoch 41/50
556/556 - 2s - loss: 0.7776 - first_order_loss: 0.6387 - second_order_loss: 0.1388
Epoch 42/50
556/556 - 2s - loss: 0.7750 - first_order_loss: 0.6391 - second_order_loss: 0.1359
Epoch 43/50
556/556 - 2s - loss: 0.7694 - first_order_loss: 0.6379 - second_order_loss: 0.1314
Epoch 44/50
556/556 - 2s - loss: 0.7712 - first_or

Epoch 20/50
556/556 - 2s - loss: 0.9108 - first_order_loss: 0.6468 - second_order_loss: 0.2640
Epoch 21/50
556/556 - 2s - loss: 0.9028 - first_order_loss: 0.6466 - second_order_loss: 0.2562
Epoch 22/50
556/556 - 2s - loss: 0.8974 - first_order_loss: 0.6460 - second_order_loss: 0.2514
Epoch 23/50
556/556 - 2s - loss: 0.8872 - first_order_loss: 0.6437 - second_order_loss: 0.2434
Epoch 24/50
556/556 - 2s - loss: 0.8820 - first_order_loss: 0.6449 - second_order_loss: 0.2371
Epoch 25/50
556/556 - 2s - loss: 0.8728 - first_order_loss: 0.6433 - second_order_loss: 0.2295
Epoch 26/50
556/556 - 2s - loss: 0.8649 - first_order_loss: 0.6417 - second_order_loss: 0.2232
Epoch 27/50
556/556 - 2s - loss: 0.8586 - first_order_loss: 0.6435 - second_order_loss: 0.2150
Epoch 28/50
556/556 - 2s - loss: 0.8527 - first_order_loss: 0.6421 - second_order_loss: 0.2106
Epoch 29/50
556/556 - 2s - loss: 0.8427 - first_order_loss: 0.6400 - second_order_loss: 0.2028
Epoch 30/50
556/556 - 2s - loss: 0.8379 - first_or